In [ ]:
import json
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# ------------------------ Book Class ------------------------
class Book:
    def __init__(self, book_id, title, author, description, times_borrowed=0):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.description = description
        self.times_borrowed = times_borrowed

# ------------------------ User Class ------------------------
class User:
    def __init__(self, user_id, name):
        self.user_id = user_id
        self.name = name
        self.borrowed_books = {}  # book_id -> due_date

    def borrow_book(self, book_id):
        due_date = datetime.now() + timedelta(days=14)
        self.borrowed_books[book_id] = due_date.strftime("%Y-%m-%d")

    def return_book(self, book_id):
        if book_id in self.borrowed_books:
            del self.borrowed_books[book_id]

    def extend_due_date(self, book_id):
        if book_id in self.borrowed_books:
            old_date = datetime.strptime(self.borrowed_books[book_id], "%Y-%m-%d")
            new_date = old_date + timedelta(days=7)
            self.borrowed_books[book_id] = new_date.strftime("%Y-%m-%d")

# ------------------------ Admin Class ------------------------
class Admin:
    def __init__(self, library):
        self.library = library

    def add_book(self, book):
        self.library.books[book.book_id] = book
        self.library.save_books()

    def add_user(self, user):
        self.library.users[user.user_id] = user
        self.library.save_users()

# ------------------------ Library Class ------------------------
class Library:
    def __init__(self):
        self.books = self.load_books()
        self.users = self.load_users()

    def load_books(self):
        if not os.path.exists("books.json"): return {}
        with open("books.json", "r") as file:
            data = json.load(file)
        return {bid: Book(bid, **b) for bid, b in data.items()}

    def save_books(self):
        with open("books.json", "w") as file:
            json.dump({bid: b.__dict__ for bid, b in self.books.items()}, file, indent=4)

    def load_users(self):
        if not os.path.exists("users.json"): return {}
        with open("users.json", "r") as file:
            data = json.load(file)
        return {uid: User(uid, u['name']) for uid, u in data.items()}

    def save_users(self):
        with open("users.json", "w") as file:
            json.dump({uid: {"name": u.name, "borrowed_books": u.borrowed_books} for uid, u in self.users.items()}, file, indent=4)

    def recommend_books(self, input_text):
        corpus = [book.description for book in self.books.values()]
        titles = [book.title for book in self.books.values()]
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(corpus + [input_text])
        cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
        top_indices = cosine_sim.argsort()[0][-3:][::-1]
        return [titles[i] for i in top_indices]

    def check_due_dates(self):
        for user in self.users.values():
            for book_id, due in user.borrowed_books.items():
                due_date = datetime.strptime(due, "%Y-%m-%d")
                if datetime.now() > due_date:
                    print(f"[Ogohlantirish] {user.name}, sizning {book_id} kitob muddati o'tgan!")

    def statistics(self):
        sorted_books = sorted(self.books.values(), key=lambda b: b.times_borrowed, reverse=True)
        print("Eng ko‘p o‘qilgan kitoblar:")
        for b in sorted_books[:3]:
            print(f"{b.title} ({b.times_borrowed} marta)")

    def cli(self):
        while True:
            print("""
1. Kitob qidirish
2. Ijaraga olish
3. Qaytarish
4. Tavsiya
5. Statistikalar
6. Chiqish
""")
            choice = input("Tanlang: ")
            if choice == '1':
                keyword = input("Kitob nomi yoki muallif: ").lower()
                for book in self.books.values():
                    if keyword in book.title.lower() or keyword in book.author.lower():
                        print(f"{book.book_id} - {book.title} by {book.author}")
            elif choice == '2':
                uid = input("Foydalanuvchi ID: ")
                bid = input("Kitob ID: ")
                user = self.users.get(uid)
                book = self.books.get(bid)
                if user and book:
                    user.borrow_book(bid)
                    book.times_borrowed += 1
                    self.save_books()
                    self.save_users()
                    print("Kitob ijaraga olindi.")
            elif choice == '3':
                uid = input("Foydalanuvchi ID: ")
                bid = input("Kitob ID: ")
                user = self.users.get(uid)
                if user:
                    user.return_book(bid)
                    self.save_users()
                    print("Kitob qaytarildi.")
            elif choice == '4':
                query = input("Qanday turdagi kitoblar qiziqtiradi? ")
                recommendations = self.recommend_books(query)
                print("Tavsiya etiladigan kitoblar:")
                for title in recommendations:
                    print(title)
            elif choice == '5':
                self.statistics()
            elif choice == '6':
                break
            else:
                print("Noto‘g‘ri tanlov")

# ------------------------ Run ------------------------
if __name__ == "__main__":
    lib = Library()
    admin = Admin(lib)
    # Misol uchun boshlang'ich foydalanuvchi va kitoblar qo‘shish (faqat bir marta)
    # admin.add_user(User("u1", "Ali"))
    # admin.add_book(Book("b1", "Python Asoslari", "Hasanov", "Python dasturlashga kirish"))

    lib.check_due_dates()
    lib.cli()



1. Kitob qidirish
2. Ijaraga olish
3. Qaytarish
4. Tavsiya
5. Statistikalar
6. Chiqish


1. Kitob qidirish
2. Ijaraga olish
3. Qaytarish
4. Tavsiya
5. Statistikalar
6. Chiqish


1. Kitob qidirish
2. Ijaraga olish
3. Qaytarish
4. Tavsiya
5. Statistikalar
6. Chiqish

Noto‘g‘ri tanlov

1. Kitob qidirish
2. Ijaraga olish
3. Qaytarish
4. Tavsiya
5. Statistikalar
6. Chiqish

Noto‘g‘ri tanlov

1. Kitob qidirish
2. Ijaraga olish
3. Qaytarish
4. Tavsiya
5. Statistikalar
6. Chiqish

